In [ ]:
testfile = r'C:\Users\cfcur\Google Drive\Sewing\Puperita\PUPERITAmodel1181MYNESTSleepSack.pdf'
patternpage = 32

In [354]:
import pikepdf

pdf = pikepdf.Pdf.open(testfile)
dest = pikepdf.Pdf.new()

copypages = []
for p in range(patternpage,patternpage+2):
    localpage = dest.copy_foreign(pdf.pages[p])
    xobj = pikepdf.Page(localpage).as_form_xobject()
    copypages.append(xobj)

mbox = localpage.MediaBox
matrix = pikepdf.PdfMatrix(xobj.Matrix)

# translate, then rotate
m = matrix.translated(-mbox[2]/2,-mbox[3]/2)
m = m.rotated(90)
m = m.translated(mbox[2]/2,mbox[3]/2)
print(m.encode())

# define the contents based on the pages first
content_txt = f"""
    q
    1 0 0 1 0 0 cm
    /Page{patternpage} Do
    1 0 0 1 {mbox[2]} 0 cm
    /Page{patternpage+1} Do
    Q"""
newpage_contents = pikepdf.Stream(dest,content_txt.encode())

xobj = pikepdf.Dictionary({})
xobj[f'/Page{patternpage}'] = copypages[0]
xobj[f'/Page{patternpage+1}'] = copypages[1]

newpage = pikepdf.Dictionary(
    Type=pikepdf.Name.Page, 
    MediaBox=[mbox[0],mbox[1],mbox[2]*2,mbox[3]],
    Resources=pikepdf.Dictionary(XObject=xobj),
    Contents=newpage_contents
)

localRoot = dest.copy_foreign(pdf.Root)
dest.Root.OCProperties = localRoot.OCProperties
dest.pages.append(newpage)
dest.save('pike-test.pdf')

# it works! layers, not too crazy, positioning, yay

b'0.000000 1.000000 -1.000000 0.000000 622.205500 97.795500'


In [ ]:
from PyPDF4 import PdfFileReader

in_doc = PdfFileReader(open(testfile,"rb"))
print(in_doc.numPages)
# encryption not supported, even though this file doesn't have password protection. Not a good sign.

In [214]:
import fitz #pymupdf, not sure why it's called fitz
src = fitz.open(testfile)

# open a second copy and delete all the pages
doc = fitz.open(testfile)
doc.deletePageRange(0,-1)

# doc = fitz.open() # blank

# copy over the layer info
# ocgs = src.get_ocgs()
# xrefmap = dict.fromkeys(ocgs.keys())

# for xref,oc in ocgs.items():
#     xrefmap[xref] = doc.add_ocg(name=oc['name'],on=oc['on'],intent=oc['intent'],usage=oc['usage'])

r = src.pageCropBox(patternpage)

# define the 4 rectangles per page
r1 = r * 0.5                           # top left rect
r2 = r1 + (r1.width, 0, r1.width, 0)   # top right
r3 = r1 + (0, r1.height, 0, r1.height) # bottom left
r4 = fitz.Rect(r1.br, r.br)            # bottom right

# put them in a list
r_tab = [r1, r2, r3, r4]

# now copy input pages to output# now copy input pages to output
page = doc.newPage(-1,r.width,r.height)

c = b""
for spage in src.pages(patternpage,patternpage+4): 
    for xref in spage._getContents():
        c += src.xrefStream(xref)
    

fitz.TOOLS._insert_contents(page, c)

# by all means, save new file using garbage collection and compression
doc.save("4up-fitz.pdf",garbage = 4, deflate = True)

b'/OC /MC0 BDC \nBT\n0.5 0.85 1 0 k\n/GS0 gs\n/TT0 1 Tf\n-0.025 Tc 12 0 0 12 10.1724 696.2686 Tm\n[(R)24.1(o'


In [122]:
src.pages(33,38)

<generator object Document.pages at 0x000002C548C3D848>